# Generating deterministic model with different revenue rates


### Running Deterministic Optimization

In [ ]:
import numpy as np
state_space = np.linspace(5,7,50)
fr_universe = list(np.exp(state_space)/1000)


In [ ]:
#fr_universe = list(np.linspace(np.exp(5)/1000,np.exp(7)/1000,50))
#state_space = np.array(np.log(fr_universe*1000))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from Logistics_stoch import CShip
from Logistics_stoch import CRoundtrip
from Logistics_stoch import CJourney
from Logistics_stoch import CPort
import deterministic_opt_func as det_opt

oJourney = CJourney(NrOfRoundtrips = 1,
                    LegsPerRoundtrip = 3,
                    OpprtCostCapitalRate = 0.08,
                    DailyHire_USDperDay = 30000,
                    FutureProfitPotential_USDperDay = 0)#12968)


oShip = CShip(Vmin=10, 
              Vmax=17, 
              DWTscantling=157880, 
              DWTdesign=145900, 
              Lightweight=49000, 
              k=0.00000391, 
              p=381, 
              g=3.1, 
              a=0.666667, 
              ShipDischargeRate=3000, 
              BallastCapacity=54500,
              MinFillRateShip=0.3,
              AuxFuelConsumption_TonnePerDay=5)

results = {}
for fr in fr_universe:
    print("Running for Freight Rate of: " + str(fr))
    
    oPortList = []

    df_data = pd.read_csv('journey_data.csv',sep=',')

    for i in range(0,oJourney.LegsPerRoundtrip+1):
        for index,row in df_data.iterrows():
            v_name = row['Variable']
            v_value = row[str(i)]
            #print("Loading from excel, leg:"+str(i)+", Executing statement("+str(v_name)+"="+str(v_value)+")")
            exec(v_name + '=' + str(v_value))


        oPort = CPort(PortNr=i, 
                      DistancePreviousPort_nm=DistancePreviousPort_nm,
                      LoadingRate_QbmetresperHr=LoadingRate_QbmetresperHr,
                      WaitingTime_Hrs=WaitingTime_Hrs,              
                      CargoIntake_Barrels=CargoIntake_Barrels,
                      CargoIntake_QBmetresperBarrel=CargoIntake_QBmetresperBarrel,
                      CargoIntake_QbmetresPerTonne=CargoIntake_QbmetresPerTonne,
                      FixedPortAccessCosts_USD = FixedPortAccessCosts_USD,
                      UnloadingCharge_USDperHr = UnloadingCharge_USDperHr,
                      LoadingCharge_USDperHr = LoadingCharge_USDperHr,
                      CargoRevenueRate_USDperBarrelper1000nm = fr, # CargoRevenueRate_USDperBarrelper1000nm,
                      MainBunkerRate_USDperBarrel = MainBunkerRate_USDperBarrel,
                      MainBunker_QBmetresperBarrel = MainBunker_QBmetresperBarrel,
                      MainBunker_QbmetresPerTonne = MainBunker_QbmetresPerTonne,
                      AuxFuelRate_USDperTonne = AuxFuelRate_USDperTonne,
                      UnloadingCosts_USD = UnloadingCosts_USD,
                      LoadingCosts_USD = LoadingCosts_USD,
                      CargoIntake_Tonne = CargoIntake_Tonne,
                      UnloadingTime_Hrs = UnloadingTime_Hrs,
                      LoadingTime_Hrs = LoadingTime_Hrs,
                      LegRevenue_Barrels = LegRevenue_Barrels,
                      LegRevenue_USD = LegRevenue_USD)

        oPortList.append(oPort)

    oRTList_det = det_opt.run_deterministic_opt(oJ=oJourney,
                                                oS=oShip,
                                                oPL=oPortList)
    results[fr] = oRTList_det
# storing solution in the journey object
oJourney.oRTList_det = oRTList_det


In [ ]:
s = np.zeros(shape=(50,5))
#for fr in fr_universe:
import matplotlib.pyplot as plt
plt.subplots(figsize=(15, 5))
plt.subplot(1, 2, 1)
determ_legs = {}
for leg_nr in range(0,oJourney.LegsPerRoundtrip):
    y = [results[fr][1].oLegList[leg_nr].Speed_kn for fr in fr_universe]
    #y = [results[fr][1].RoundtripGoodwill for fr in fr_universe]
    x = [state_space[fr_universe.index(fr)] for fr in fr_universe]
    x = [fr for fr in fr_universe]
    determ_legs[leg_nr] = (x,y)
    plt.plot(x, y, label='Leg ' + str(leg_nr+1))
    #s[fr_universe.index(fr),leg_nr] = results[fr][1].oLegList[leg_nr].Speed_kn

rt_y = [results[fr][1].RoundtripGoodwill for fr in fr_universe]
rt_x = [fr for fr in fr_universe]


#plt.plot(state_space,s)
plt.xlabel('Cargo Revenue Rate (USD per Barrel per 1000nm)') 
plt.ylabel('nm/h') 
plt.title('Speeds of the vessel')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(rt_x,rt_y,label='Roundtrip profit')
plt.xlabel('Cargo Revenue Rate (USD per Barrel per 1000nm)') 
plt.ylabel('US$') 
plt.title('Roundtrip profit')
plt.legend()

plt.show()


## Running the stochastic optimization

In [ ]:
import OU_process
ou_process = OU_process.OU_process(r_bar = 6.2, 
                                   r_lambda=0.0, 
                                   r_sigma=0.3, 
                                   r_start=5.4,
                                   r_end=6.9                              
                                  )

In [ ]:
from Optimization_Problem import Optimization_Problem
import FFA
ffa=FFA.FFA(fut_curve_slope = -0.01)
eta=0
op0 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=ou_process, 
                          ffa=ffa, 
                          eta=eta) # determines how important is the variance of future spot rates)
oRTList0 = op0.run_optimization()

# Display the results



## Value Function ($G$) or Goodwill 

The final value function after all legs and rountrip were calculated from the back.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter 
x = np.exp(oRTList0[oJourney.NrOfRoundtrips].RoundtripGoodwill.x)/1000
plt.plot(rt_x,
         oRTList0[oJourney.NrOfRoundtrips].RoundtripGoodwill(np.log(np.array(rt_x)*1000)),
        label='Stochastic')
plt.plot(rt_x,
         rt_y,
         label='Deterministic')

plt.xlabel('State of Freight Market') 
plt.ylabel('Goodwill') 
plt.title('The value of the ship')
plt.legend()
plt.gca().yaxis.set_minor_formatter(NullFormatter())
plt.show()



## Speed trip

In [ ]:
plt.subplots(figsize=(15, 10))
x = np.exp(oRTList0[1].oLegList[0].Speed_kn.x)/1000
for leg_i in range(0,oJourney.LegsPerRoundtrip):
    plt.plot(x,
             oRTList0[1].oLegList[leg_i].Speed_kn.y, 
             label='Stoch Leg ' + str(leg_i+1),
             dashes=[6, 2])
    (x1,y1) = determ_legs[leg_i]
    plt.plot(x1,
             y1, 
             label='Determ Leg ' + str(leg_i+1))
    
plt.xlabel('State - Cargo Revenue Rate (USD per Barrel per 1000nm)') 
plt.ylabel('nm/h') 
plt.title('Speeds of the vessel')
plt.legend()
plt.show()


In [ ]:
for j in range(2-1, -1, -1):
    print(j)

## Hedge Ratio

In [ ]:
plt.plot(oRTList0[1].oLegList[0].Hedge_Ratio.x,
         oRTList0[1].oLegList[0].Hedge_Ratio.y, 
         label='Leg 5')
plt.plot(oRTList0[1].oLegList[1].Hedge_Ratio.x,
         oRTList0[1].oLegList[1].Hedge_Ratio.y, 
         label='Leg 4')
plt.plot(oRTList0[1].oLegList[2].Hedge_Ratio.x,
         oRTList0[1].oLegList[2].Hedge_Ratio.y, 
         label='Leg 3')
plt.plot(oRTList0[1].oLegList[3].Hedge_Ratio.x,
         oRTList0[1].oLegList[3].Hedge_Ratio.y, 
         label='Leg 2')
plt.plot(oRTList0[1].oLegList[4].Hedge_Ratio.x,
         oRTList0[1].oLegList[4].Hedge_Ratio.y, 
         label='Leg 1')
plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Optimal hedge ratio for each state')
plt.show()

## Changing parameters of the problem:

Assume that the forward rates are discounted by more than previously assumed, in this case hadging is more costly on average, we expect less hedging and decrease of the hedge ratio:

> ffa = FFA.FFA(fut_curve_slope = -0.01)

In [ ]:
from Optimization_Problem import Optimization_Problem
op1 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, 
                                                        r_lambda=0, 
                                                        r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.02), 
                          eta=0.000001) 
oRTList1 = op1.run_optimization()

## Comparison of hedge ratios

In [ ]:
plt.plot(oRTList0[1].oLegList[0].Hedge_Ratio.x,
         oRTList0[1].oLegList[0].Hedge_Ratio.y, 
         label='Discount=-1% leg=4')
plt.plot(oRTList0[1].oLegList[4].Hedge_Ratio.x,
         oRTList0[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% leg=5')

plt.xlabel('State of Freight Market') 
#plt.ylabel('Optimal Hedge Ratio') 
plt.title('Optimal hedge ratio for each state')
plt.legend(loc='lower right')
plt.show()

## Comparison of time of journeys of the last leg

In [ ]:
plt.plot(oRTList0[1].oLegList[0].TimeAtSea_Days.x,
         oRTList0[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-1%')
plt.plot(oRTList1[1].oLegList[0].TimeAtSea_Days.x,
         oRTList1[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-2%')
plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Time of trip in days')
plt.show()

## More importance to the volatility of the spot rates 
>eta=0.000001

In [ ]:
op2 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=3, r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.02), 
                          eta=0.0000015) 
oRTList2 = op2.run_optimization()

In [ ]:
plt.plot(oRTList1[1].oLegList[4].Hedge_Ratio.x,
         oRTList1[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% eta=1e-6')
plt.plot(oRTList2[1].oLegList[4].Hedge_Ratio.x,
         oRTList2[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-2% eta=1e-6')
plt.plot(oRTList1[1].oLegList[4].Hedge_Ratio.x,
         oRTList1[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-2% eta=1.5e-6')

plt.xlabel('State of Freight Market') 
#plt.ylabel('Optimal Hedge Ratio') 
plt.title('Optimal hedge ratio for each state')
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(oRTList0[1].oLegList[0].TimeAtSea_Days.x,
         oRTList0[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-1% eta=1e-6')
plt.plot(oRTList1[1].oLegList[0].TimeAtSea_Days.x,
         oRTList1[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-2% eta=1e-6')
plt.plot(oRTList2[1].oLegList[0].TimeAtSea_Days.x,
         oRTList2[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-2% eta=1.5e-6')

plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Time of trip in days')
plt.show()

## Changing paramters of the OU process

> ou_process = OU_process.OU_process(r_bar = 6.2, r_lambda=3, r_sigma=0.3)


In [ ]:
op3 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=0.5, r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.05), 
                          eta=0.000001) 
oRTList3 = op3.run_optimization()

op4 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=0.8, r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.05), 
                          eta=0.000001) 
oRTList4 = op4.run_optimization()

In [ ]:
plt.plot(oRTList3[1].oLegList[4].Hedge_Ratio.x,
         oRTList3[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% eta=5e-7 r_bar=6.8')
plt.plot(oRTList4[1].oLegList[4].Hedge_Ratio.x,
         oRTList4[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% eta=5e-7 r_bar=6.2')


plt.xlabel('State of Freight Market') 
#plt.ylabel('Optimal Hedge Ratio') 
plt.title('Optimal hedge ratio for each state')
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.plot(oRTList3[1].oLegList[4].TimeAtSea_Days.x,
         oRTList3[1].oLegList[4].TimeAtSea_Days.y, 
         label='Leg 5 Discount=-1% eta=1e-7 r_bar=6.2')
plt.plot(oRTList4[1].oLegList[4].TimeAtSea_Days.x,
         oRTList4[1].oLegList[4].TimeAtSea_Days.y, 
         label='Leg 5 Discount=-1% eta=1e-7 r_bar=6.8')

plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='lower left')
plt.title('Time of trip in days')
plt.show()

## Checking if the function is convex

In [ ]:
ou_process = OU_process.OU_process(r_bar = 6.2, r_lambda=1.8, r_sigma=0.3)
ffa = FFA.FFA(fut_curve_slope = -0.5)

params = [ou_process.r_bar ,oShip, oPortList, oJourney, ou_process, ffa, 0.00002]

In [ ]:
leg = oRTList0[1].oLegList[0]
t_range = np.linspace(leg.TimeAtSeaMin_Days,leg.TimeAtSeaMax_Days,num=100)
h_range = np.linspace(0,1,num=100)
z = np.zeros((100,100))
it = 0
ih = 0
max_z = 0
for t in t_range:
    ih = 0
    for h in h_range:
        z[it,ih],q = leg.CalcGoodwill([h,t],params)
        if z[it,ih]>max_z:
            h_max = h
            t_max = t
            max_z = z[it,ih] 
        ih = ih + 1
    it = it + 1
    

In [ ]:
print([h_max,t_max,max_z])

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
# Read data from a csv
x, y = t_range, h_range
fig = go.Figure(data=[go.Surface(z=z.T, x=x, y=y)])
fig.update_layout(title='Goodwill as a function of time and hedge ratio', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                    xaxis_title='Time of the leg in days',
                    yaxis_title='Hedge ratio from 0 to 1',
                    zaxis_title='Value of the ship'),
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
leg.FindBestGoodwill(ou_process, ffa, 0, oJourney, oShip, oRTList0, oPortList)

In [ ]:
plt.plot(leg.Hedge_Ratio.x,leg.Hedge_Ratio.y)

In [ ]:
leg = oRTList0[1].oLegList[0]
leg_d = results[0.41174662127882383][1].oLegList[0]
s1_a = []
s2_a = []

x = np.linspace(10,leg.TimeAtSeaMax_Days,num=70)

for t in x:
    s1 = leg.CalcGoodwill([0,t], [r,oShip,oPortList,oJourney,ou_process,ffa,eta])
    s1_a.append(s1)
    s2 = leg_d.CalcGoodwill(t, oJourney, oShip, results[0.41174662127882383], oPortList)
    s2_a.append(s2)
    #print("Stochastic:" + str(s1))
    #print("Deterministic:" + str(s2))
    #print(s1[0]-s2)
plt.plot(x,s1_a)
plt.show()
plt.plot(x,s2_a)
plt.show()


#np.log(np.array(list(results.keys()))*1000)
#np.exp(6.02040816)/1000


In [ ]:
t1 = leg.TimeAtSea_Days(6.02040816)
t2 = leg_d.TimeAtSea_Days


In [ ]:

#plt.plot(np.array(s1_a)[10:30,0])
#plt.plot(s2_a[10:30])
r = 6.02040816

df1 = np.exp(-1 * oJourney.OpprtCostCapitalRate * (t1 / 365))
df2 = np.exp(-1 * oJourney.OpprtCostCapitalRate * (t2 / 365))


states_next , step_next = ou_process.gen_cond_states(r,leg.TimeAtSeaMax_Days,10e+100, -10e+100)
prob_space = ou_process.p_vector(states_next, step_next, r, t1)

print(np.dot(prob_space,states_next))
#leg.FutureGoodwill(states_next)

plt.plot(states_next,leg.FutureGoodwill(states_next))
print("Prob Space:" + str(np.sum(prob_space)))
print("Deterministic:" + str(df2 * leg_d.FutureGoodwill))
print("Stochastic:" + str(df1 * np.dot(prob_space,leg.FutureGoodwill(states_next))))
print("Difference:" + str(df1 * np.dot(prob_space,leg.FutureGoodwill(states_next))-df1 * leg_d.FutureGoodwill))
print("Risk Neutral Adjustment:" + str(df1 * np.exp(0*0.5*0.25*0.25*t1/365 + np.dot(prob_space,np.log(leg.FutureGoodwill(states_next))))-df1 * leg_d.FutureGoodwill))

#plt.plot(np.exp(leg.FutureGoodwill.x)/1000,leg.FutureGoodwill.y)
#plt.show()

#plt.plot(np.exp(leg.FutureGoodwill.x)/1000,leg.FutureGoodwill.y)
#plt.show()

# actual difference:
#print(np.array(s1_a)[indx,0] - np.array(s1_a)[indx,1])
